In [1]:
from __future__ import division, print_function, absolute_import

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import glob
import numpy as np
import os

pwd = os.getcwd()
tfrecord_file_path = pwd + "/*.tfrecord"
filename_queue = tf.train.string_input_producer(
                              tf.train.match_filenames_once(tfrecord_file_path),
                              shuffle=True, num_epochs=None)

reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)
features = tf.parse_single_example(serialized_example,
                                       features={'data': tf.FixedLenFeature([311], tf.float32)})  
fft_batch = tf.train.batch([features['data']],batch_size=10,num_threads=1)

# Training Parameters
learning_rate = 0.01
num_steps = 200000
display_step = 1000

# Network Parameters
num_hidden_1 = 64 # 1st layer num features
num_input = 311

X = tf.placeholder("float", [None, num_input],name='input_x')

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b1': tf.Variable(tf.random_normal([num_input])),
}

# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    return layer_1


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    return layer_1

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
loss = tf.reduce_sum(tf.pow(y_true - y_pred, 2))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

saver = tf.train.Saver()
tf.add_to_collection('output_y', decoder_op)
tf.add_to_collection('output_y', loss)

with tf.Session() as sess:
    
    tf.local_variables_initializer().run()
    tf.global_variables_initializer().run()
    coord = tf.train.Coordinator()
    thread = tf.train.start_queue_runners(sess=sess,coord=coord)
    
    for i in range(1,num_steps+1):
        _, l = sess.run([optimizer, loss],feed_dict={X: fft_batch.eval()})
        if i % display_step == 0 or i == 1:
            print('Step %i: Minibatch Loss: %f' % (i, l))
    
    saver.save(sess,'model/fft')
    
    coord.request_stop()
    coord.join(thread)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Step 1: Minibatch Loss: 974.832703
Step 1000: Minibatch Loss: 6.661167
Step 2000: Minibatch Loss: 6.102637
Step 3000: Minibatch Loss: 7.391111
Step 4000: Minibatch Loss: 5.582391
Step 5000: Minibatch Loss: 5.580720
Step 6000: Minibatch Loss: 5.833238
Step 7000: Minibatch Loss: 7.480004
Step 8000: Minibatch Loss: 4.152396
Step 9000: Minibatch Loss: 4.692162
Step 10000: Minibatch Loss: 3.909145
Step 11000: Minibatch Loss: 2.580040
Step 12000: Minibatch Loss: 3.467189
Step 13000: Minibatch Loss: 2.563790
Step 14000: Minibatch Loss: 3.732518
Step 15000: Minibatch Loss: 3.898960
Step 16000: Minibatch Loss: 4.128016
Step 17000: Minibatch Loss: 4.310999
Step 18000: Minibatch Loss: 4.404308
Step 19000: Minibatch Loss: 3.749220
Step 20000: Minibatch 

Step 195000: Minibatch Loss: 1.104366
Step 196000: Minibatch Loss: 1.167112
Step 197000: Minibatch Loss: 1.769822
Step 198000: Minibatch Loss: 1.570932
Step 199000: Minibatch Loss: 1.597723
Step 200000: Minibatch Loss: 1.813080
